<a href="https://colab.research.google.com/github/zhr-frj/palCodedInterpreter/blob/main/pal_PythonEnterpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install bitsandbytes accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.2 MB/s eta 0:00:00


In [2]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re

In [3]:
model_id = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

In [4]:
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)

Device set to use cuda:0


In [18]:
# def create_prompt_two_section(question, observation=None):
#     prompt = (
#         "You must output EXACTLY two sections separated by the token '---SEPARATOR---'.\n"
#         "SECTION A (CODE): Only valid Python code. No comments, no text, no backticks, no labels like 'Result:', 'Answer:', 'Step:'. "
#         "Perform exact numerical calculations. Do NOT round intermediate steps. "
#         "Always carefully account for units and quantities. For example, if a problem mentions "
#         "damaged boxes, each box may contain multiple items—do NOT subtract the number of boxes directly, "
#         "subtract the total items in those boxes. "
#         "If you use any external Python module such as math, ALWAYS include the required import "
#         "statement at the top (e.g., 'import math'). Never assume modules are preloaded. "
#         "For motion problems with two moving objects (speed/distance/time), "
#         "use the relative speed method: time to meet = initial distance / relative speed. "
#         "Do NOT alter numeric values from the question; use them exactly as stated. "
#         "Store the final numeric result in variable 'result' and end with print(result).\n"
#         "SECTION B (EXPLANATION): Human-readable explanation in Persian.\n\n"
#         "FORMAT:\n"
#         "SECTION A (CODE)\n"
#         "---SEPARATOR---\n"
#         "SECTION B (EXPLANATION)\n\n"
#     )
#     if observation:
#         prompt += f"OBSERVATION_DATA: {observation}\n\n"

#     prompt += (
#         f"Question: {question}\n"
#         "Important: Ensure calculations are precise and final answer is exact. "
#         "Do not make assumptions; always calculate based on the quantities explicitly mentioned in the problem. "
#         "If a module is required (e.g., for ceiling, logarithm, or advanced math), "
#         "import it explicitly inside SECTION A.\n"
#         "SECTION A MUST contain only Python code. No text, labels, or comments. "
#         "If any non-Python text appears, treat it as:\n"
#         "Observation: Execution Error: invalid code in SECTION A\n"
#         "Final Answer: invalid code in SECTION A\n"
#     )
#     return prompt



def create_prompt_two_section(question, observation=None):
    prompt = (
        "You must output EXACTLY two sections separated by the token '---SEPARATOR---'.\n"
        "SECTION A (CODE): Only valid Python code. No comments, no text, no backticks, no labels like 'Result:', 'Answer:', 'Step:'. "
        "Perform exact numerical calculations. Do NOT round intermediate steps. "
        "Always carefully account for units and quantities. For example, if a problem mentions "
        "damaged boxes, each box may contain multiple items—do NOT subtract the number of boxes directly, "
        "subtract the total items in those boxes. "
        "If you use any external Python module (e.g., math, decimal, numpy), ALWAYS include the required import "
        "statement at the top of SECTION A. Do NOT assume any module is preloaded. "
        "For motion problems with two moving objects (speed/distance/time), "
        "use the relative speed method: time to meet = initial distance / relative speed. "
        "Do NOT alter numeric values from the question; use them exactly as stated. "
        "Store the final numeric result in variable 'result' and end with print(result).\n"
        "SECTION B (EXPLANATION): Human-readable explanation in Persian.\n\n"
        "FORMAT:\n"
        "SECTION A (CODE)\n"
        "---SEPARATOR---\n"
        "SECTION B (EXPLANATION)\n\n"
    )
    if observation:
        prompt += f"OBSERVATION_DATA: {observation}\n\n"

    prompt += (
        f"Question: {question}\n"
        "Important: Ensure calculations are precise and final answer is exact. "
        "Do not make assumptions; always calculate based on the quantities explicitly mentioned in the problem. "
        "If any module is required for functions such as ceiling, logarithm, square root, or array operations, "
        "import it explicitly inside SECTION A. "
        "Always include all imports at the very top of SECTION A before using any functions from them. "
        "SECTION A MUST contain only Python code. No text, labels, or comments. "
        "If any non-Python text appears, treat it as:\n"
        "Observation: Execution Error: invalid code in SECTION A\n"
        "Final Answer: invalid code in SECTION A\n"
    )
    return prompt

In [19]:
def extract_code(text):
    match = re.search(r"Action:\n([\s\S]+)", text)
    if match:
        code = match.group(1).strip()
        lines = code.splitlines()
        code_lines = [line for line in lines if line.strip() and not line.strip().startswith("#")]
        return "\n".join(code_lines)
    return ""

In [20]:
def run_code(code):
    exec_env = {}
    try:
        exec(code, {}, exec_env)
    except Exception as e:
        print("Execution error:", e)
        return f"Execution Error: {e}"
    if "result" in exec_env:
        return exec_env["result"]
    nums = [v for v in exec_env.values() if isinstance(v, (int, float))]
    if nums:
        return nums[-1]
    return "No result variable found."


In [21]:
def pal_chain(question, max_steps=3):
    observation = None
    for step in range(1, max_steps + 1):
        print(f"\n--- Step {step} ---")
        prompt = create_prompt(question, observation)
        output = llm(prompt)[0]
        generated = output.get('generated_text', '') or output.get('text', '')
        code = extract_code(generated)
        if not code:
            print("No code extracted.")
            return "No code generated."
        print("Generated code:\n", code)
        observation = run_code(code)
        print("Observation:", observation)
        if isinstance(observation, (int, float, str)):
            break
    return observation


In [22]:

question = "Reza has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"

final_answer = pal_chain(question)
print("\n✅ Final Answer:", final_answer)


--- Step 1 ---


NameError: name 'create_prompt' is not defined

In [15]:
question = "John has 4 boxes of pencils. Each box contains 10 pencils. He finds 7 more pencils in his drawer. How many pencils does he have now?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---


NameError: name 'create_prompt' is not defined

In [17]:
question = "Lily read 12 pages on Monday, 15 pages on Tuesday, and twice as many pages on Wednesday as on Monday. How many pages did she read in total?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---


NameError: name 'create_prompt' is not defined

In [ ]:
question = "A box contains 8 markers. A teacher buys 6 boxes. If she already had 5 markers, how many markers does she have in total?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 result = 8 * 6 + 5
print(result)
53
Observation: 53

✅ Final Answer: 53


In [ ]:
question = "A car travels 60 km in one hour. How far will it travel in 3 hours?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 Result:
result = 60 * 3
print(f"The car will travel {result} kilometers in 3 hours.")
Execution error: invalid syntax (<string>, line 1)
Observation: Execution Error: invalid syntax (<string>, line 1)

✅ Final Answer: Execution Error: invalid syntax (<string>, line 1)


In [ ]:
question = "Each notebook costs 4 dollars. Ali buys 7 notebooks. How much does he pay?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 result = 4 * 7
print(f"Ali pays ${result} for buying 7 notebooks.")
result = 4 * 7
print(f"Ali pays ${result} for buying 7 notebooks.")
Ali pays $28 for buying 7 notebooks.
Ali pays $28 for buying 7 notebooks.
Observation: 28

✅ Final Answer: 28


In [ ]:
question = "A farmer has 56 apples and wants to divide them equally into 7 baskets. How many apples will be in each basket?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 result = 56 // 7
print(f"Each basket will contain {result} apples.")
Each basket will contain 8 apples.
Observation: 8

✅ Final Answer: 8


In [ ]:
question = "A factory produces 120 bottles of juice per hour. In 6 hours, how many bottles are produced? If 2 trucks each carry 350 bottles, how many bottles remain at the factory after loading the trucks?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 hourly_production = 120
bottles_produced_in_6_hours = hourly_production * 6
truck_capacity = 350
bottles_per_truck = truck_capacity
total_bottles_loaded = bottles_per_truck * 2
Observation: 700

✅ Final Answer: 700


In [ ]:

question = "In a class of 40 students, 60% are boys. If 5 boys and 3 girls leave the class, what is the new percentage of boys?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 num_boys = int(0.6 * 40)
num_girls = 40 - num_boys
num_boys -= 5
num_girls -= 3
new_num_boys = num_boys
new_num_girls = num_girls
new_total = new_num_boys + new_num_girls
new_percentage = new_num_boys / new_total * 100
result = new_percentage
print
Observation: 59.375

✅ Final Answer: 59.375


In [ ]:
question = "A bookstore sold 12 novels on Monday, twice as many on Tuesday, and 3 fewer than Tuesday on Wednesday. How many novels were sold in total during these three days?"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)

In [ ]:

question = "جان ۵ مداد به قیمت هر کدام ۲ دلار و ۳ دفترچه به قیمت هر کدام ۵ دلار خریداری می‌کند. سپس ۴ دلار تخفیف می‌گیرد. او در نهایت چقدر می‌پردازد؟"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)



--- Step 1 ---
Generated code:
 num_books = 5
book_cost = 2
num_notebooks = 3
notebook_cost = 5
total_cost_before_discount = (num_books * book_cost) + (num_notebooks * notebook_cost)
discount = 4
total_cost_after_discount = total_cost_before_discount - discount
result = total_cost_after_discount
print(f"John spends {result} dollars.")
John spends 21 dollars.
Observation: 21

✅ Final Answer: 21


In [ ]:
question = "یک کشاورز ۳۶۰ سیب دارد. می‌خواهد آنها را در جعبه‌هایی که هر کدام ۱۵ سیب دارند بسته‌بندی کند. اگر ۸ جعبه خراب شوند، چند سیب فروخته می‌شود؟"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 number_of_boxes = math.ceil(360 / 15)
apples_in_good_boxes = number_of_boxes * 15
number_of_lost_boxes = 8
apples_in_lost_boxes = number_of_lost_boxes * 15
remaining_apples = 360 - apples_in_good_boxes - apples_in_lost_boxes
result = remaining_apples
Execution error: name 'math' is not defined
Observation: Execution Error: name 'math' is not defined

✅ Final Answer: Execution Error: name 'math' is not defined


In [ ]:
question = "یک دستور پخت برای هر ۲ فنجان شکر، ۳ فنجان آرد نیاز دارد. اگر ۱۸ فنجان آرد و ۲۰ فنجان شکر داری، چند دستور کامل می‌توان درست کرد و چند فنجان شکر باقی می‌ماند؟"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 result = 0
total_sugar = 20
total_water = 18
batch_sugar = 2
batch_water = 3
num_batches_sugar = total_sugar // batch_sugar
remaining_sugar = total_sugar % batch_sugar
num_batches_water = total_water // batch_water
remaining_water = total_water % batch_water
total_
Execution error: name 'total_' is not defined
Observation: Execution Error: name 'total_' is not defined

✅ Final Answer: Execution Error: name 'total_' is not defined


In [ ]:
question = "یک قطار با سرعت ۶۰ کیلومتر در ساعت، ۱۲۰ کیلومتر مسیر طی می‌کند. قطار دوم یک ساعت دیرتر در همان مسیر با سرعت ۸۰ کیلومتر در ساعت حرکت می‌کند. بعد از چند ساعت قطار دوم به اول می‌رسد؟"
final_answer = pal_chain(question)
print('\n✅ Final Answer:', final_answer)


--- Step 1 ---
Generated code:
 train_one_hour = 60 # kilometers per hour for train one
train_two_hour = 80 # kilometers per hour for train two
train_one_distance = train_one_hour * 120 # kilometers
train_two_distance = train_two_hour * 360 # kilometers (since it's moving for one hour more)
remaining_distance = 120 * 2 - train_one_distance
hours_difference = remaining_distance / train_
Execution error: name 'train_' is not defined
Observation: Execution Error: name 'train_' is not defined

✅ Final Answer: Execution Error: name 'train_' is not defined
